In [ ]:
## Based on https://dhi.github.io/mikeio/examples/Dfsu-2D-interpolation.html

## Import Libraries
import os
import numpy as np
import pandas as pd
import mikeio
import rasterio
from rasterio.transform import from_origin

## Define input and output file paths and names.
input_path = r"C:\INPUT_PATH"
input_filename =r"Area_Max.dfsu"
output_path = r"C:\OUTPUT_PATH"
output_filename = r"RainyDay_Full.tif" # This is the dfs2 converted to tif
output_filename_threshold = r"RainyDay_clip.tif" # This is the clipped raster removing values below specified threshold

## Concatenation
input_file = rf"{input_path}\{input_filename}"
output_file = rf"{output_path}\{output_filename}"
output_file_threshold =rf"{output_path}\{output_filename_threshold}"

# Check if the input file exists
if os.path.exists(input_file):
    print("The file exists.")
else:
    print("The file or path does not exist.")

## MIKE File read, must specify the variable name 
ds = mikeio.read(input_file, items="Maximum water depth")
da = ds.Maximum_water_depth

## Get an overset grid covering the domain. Specify dx. In this example dx=2.0
g = da.geometry.get_overset_grid(dx=2.0)
## Interpolate grid (This step takes some time to run)
da_grid = da.interp_like(g)
print(f'Grid created')

## Export Raster using rasterio library

with rasterio.open(
     fp=output_file,
     mode='w',
     driver='GTiff',
     height=g.ny,
     width=g.nx,
     count=1,
     dtype=da.dtype,
     crs="EPSG:2193", # adjust accordingly for projected coordinate systems. This works fine with the NZGD2000 from ARCGIS
     transform=from_origin(g.bbox.left, g.bbox.top, g.dx, g.dy)
     ) as dst:
        dst.write(np.flipud(da_grid[0].to_numpy()), 1)  

## Function to Delete values < 0.05
def apply_threshold(input_raster_path, output_raster_path, threshold):
    with rasterio.open(input_raster_path) as src:
        raster_array = src.read(1)
        transform = src.transform
        crs = src.crs
        raster_array[raster_array < threshold] = np.nan
        with rasterio.open(
            output_raster_path, 'w',
            driver='GTiff',
            height=raster_array.shape[0],
            width=raster_array.shape[1],
            count=1,
            dtype='float32',
            crs=crs,
            transform=transform,
            nodata=np.nan
        ) as dst:
            dst.write(raster_array, 1)

# Function call
#cell_size = 2
apply_threshold(output_file, output_file_threshold, 0.05) # Delete this row if you don't want to remove values. Threshold is defined as 5 cm.
print(f'Clipped raster saved in : {output_file_threshold}') # This line is just to be aware that the script has ended.